In [4]:
import pschitt.emission as em
import pschitt.vizualisation as viz
import pschitt.sky_objects as sky
import pschitt.geometry as geo
import pschitt.camera_image as ci
import numpy as np
import matplotlib.pyplot as plt
import math
import pschitt.dataset as ds
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import bokeh
from bokeh.io import output_notebook
from bokeh import plotting
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, ColorBar
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

In [5]:
import matplotlib
%matplotlib inline
output_notebook()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

font = {'size'   : 20}
matplotlib.rc('font', **font)

Loading BokehJS ...

In [3]:
shower = sky.shower()

shower.impact_point = np.array([600,600,0])

#shower direction
shower.alt = math.radians(70)
shower.az = math.radians(0)

#pointing direction off-axis
talt = math.radians(70)
taz = math.radians(0)

#shower parameters
shower_top = 12000
shower_length = 8000
shower_width = 200

shower.number_of_particles = int(1e3)

shower.gaussian_ellipsoide(shower_top, shower_length, shower_width)

tel_normal = geo.altaz_to_normal(talt,taz)
# tel1 = geo.Telescope([0,0,0],tel_normal)
# tel2 = geo.Telescope([200,0,0],tel_normal)
# tel3 = geo.Telescope([400,0,0],tel_normal)
# tel4 = geo.Telescope([800,0,0],tel_normal)
# tel5 = geo.Telescope([0,400,0],tel_normal)

# alltel = [tel1, tel2, tel3, tel4, tel5]

alltel = geo.load_telescopes(ds.get('CTA_paranal.txt'), normal=tel_normal)

trigger_intensity = 20.
noise = 0

shower.particles_angular_emission_profile = em.angular_profile.lgdt06
shower.particles_angular_emission_profile_kwargs = {'eta':1}

ci.array_shower_imaging(shower, alltel, noise)

100 telescopes loaded


In [7]:
tel = alltel[10]
x = tel.pixel_tab[:,0]
y = tel.pixel_tab[:,1]
signal = tel.signal_hist

plot = plotting.figure(plot_height=400, plot_width=400,
              match_aspect=True)

source = ColumnDataSource(dict(x=x,y=y,signal=signal))

mapper = linear_cmap(field_name='signal', palette=Spectral6, low=min(signal), high=max(signal))
plot.circle(x='x', y='y', color=mapper, fill_alpha=1, size=6, source=source)

show(plot)

In [8]:
tel_id = 10
tel = alltel[tel_id]
x = tel.pixel_tab[:,0]
y = tel.pixel_tab[:,1]
signal = tel.signal_hist 


plot = plotting.figure(plot_height=400, plot_width=400,
              match_aspect=True)

mapper = linear_cmap(field_name='y', palette=Spectral6, low=min(signal), high=max(signal))
r = plot.circle(x, y, fill_alpha=1, size=6)


def update(tel_id):
    tel = alltel[tel_id]
    x = tel.pixel_tab[:,0]
    y = tel.pixel_tab[:,1]
    signal = tel.signal_hist
    
    r.data_source.data['x'] = x
    r.data_source.data['y'] = y


show(plot, notebook_handle=True)

interact(update, tel_id=(0, len(alltel)-1, 1))

interactive(children=(IntSlider(value=49, description='tel_id', max=99), Output()), _dom_classes=('widget-inte…

<function __main__.update>

In [14]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

tel_id = 0
x = alltel[tel_id].pixel_tab[:,0]
y = alltel[tel_id].pixel_tab[:,1]

source = ColumnDataSource(dict(x=x,y=y,signal=signal))
mapper = linear_cmap(field_name='signal', palette=Spectral6, low=min(signal), high=max(signal))
p = figure(title="Camera", plot_height=600, plot_width=600, match_aspect=True)
r = p.circle(x='x', y='y', fill_alpha=1, size=10, color=mapper, source=source)
# r = p.line(x, y, color="#2222aa", line_width=3)

def update(w=1):
    source = ColumnDataSource(dict(x=alltel[w].pixel_tab[:,0],
                                   y=alltel[w].pixel_tab[:,1],
                                   signal=alltel[w].signal_hist))
    mapper = linear_cmap(field_name='signal', palette=Spectral6, low=min(signal), high=max(signal))

    r.data_source.data['x'] = alltel[w].pixel_tab[:,0]
    r.data_source.data['y'] = alltel[w].pixel_tab[:,1] #A * func(w * x + phi)
    push_notebook()
    
show(p, notebook_handle=True)

interact(update,  w=(0,99))

Loading BokehJS ...

interactive(children=(IntSlider(value=1, description='w', max=99), Output()), _dom_classes=('widget-interact',…

<function __main__.update>